In [28]:
import pandas as pd
from tqdm import tqdm

In [22]:
# NE TAG에 대한 딕셔너리 형성 {TAG_NAME : [엔티티1, 엔티티2, 엔티티3, ...]}
ne_tag_dict = pd.read_csv('../data/ne_tag_dict.csv')

ne_dict = dict()
ne_dict['VACC-B'] = list(ne_tag_dict['VACC'][ne_tag_dict['VACC'].notna()].values)
ne_dict['DIS-B'] = list(ne_tag_dict['DIS'][ne_tag_dict['DIS'].notna()].values)
ne_dict['SYMP-B'] = list(ne_tag_dict['SYMP'][ne_tag_dict['SYMP'].notna()].values)


In [32]:
# 뉴스 기사 불러오기
with open('../data/sentence_split.txt', encoding='utf-8') as f:
    news = f.read()
    
news = news.replace('.\n', ' .\n').splitlines()
news[:10]

['전두환 전 대통령이 코로나19 화이자 백신을 접종한 뒤에 건강이 급격히 악화됐다는 전 씨 측 주장이 제기된 가운데 이에 대해 이재갑 한림대 감염내과 교수는 "가당치도 않은 주장"이라고 지적하고 나섰습니다 .',
 '이재갑 한림대 감염내과 교수는 25일 CBS 라디오 \'김현정의 뉴스쇼\'와의 인터뷰에서 전 씨 사망 원인으로 코로나 백신 부작용이 언급된 것에 대해 "사실 가당치도 않은 주장"이라며 "최근에 혈액종양내과 교수님들이 여러 번 말씀드리긴 했는데 혈액암, 백혈병하고 코로나 백신하고 관련이 없다"고 선을 그었습니다 .',
 '이 교수는 "전 세계적으로 관련이 증명된 사례는 전혀 없는 상황이고 다른 백신에서도 사례가 전혀 없다"며 "지금까지는 인과관계가 없다"고 강조했습니다 .',
 '특히 "전 전 대통령이 걸린 만성골수성백혈병 같은 경우는 원래 꽤 오래 전부터 시작되면서 서서히 시작되는 백혈병 중에 하나이기 때문에 그럴 가능성은 훨씬 더 떨어진다고 볼 수 있다"고 전했습니다. \'백혈병이 급성으로 오기도 하지 않느냐\'는 진행자의 질문에는 "급성 백혈병도 현재까지 인과관계가 증명된 사례는 없다"고 전 씨의 사망 원인에 백신 부작용이 있다는 주장에 대해 다시 한 번 선을 그었습니다 .',
 '한편, 전 씨는 지난 23일 오전 8시 40분쯤 연희동 자택에서 쓰러져 숨졌습니다 .',
 '이에 대해 전 씨 최측근 민정기 전 청와대 공보비서관은 "화이자 백신 접종 후 혈액암에 걸렸으며 부작용이 의심된다"는 주장을 펼쳤습니다 .',
 '민 전 비서관은 "지난 6, 7월쯤 찾아뵀는데 굉장히 수척해진 모습으로 건강이 좋아 보이지 않았다"며 "부인 이순자 여사께 ‘왜 그러시느냐’고 물었더니 ‘백신 맞은 후 식사를 잘 못 하셔서 체중이 10㎏가량 줄었다’더라"고 말했습니다 .',
 '이후 손영래 보건복지부 중앙사고수습본부 사회전략반장은 24일 정례브리핑에서 관련 질문을 받은 후 "만약 예방 접종 후 영향이 있는 문제라면 이상 반응 신고 등의 절차를 거쳐 조사하게 되겠지

In [33]:
# 딕셔너리 매칭을 통해 수동 태깅 전 사전 태깅작업 진행
news_sents_tagged = []

for sent in tqdm(news):
    tag_lst = []
    token_lst = sent.replace('\"', '').replace("\'", "").split() # 따옴표, 쌍따옴표 기호 제거
    
    for token in token_lst:
        if token in ne_dict['VACC-B']:
            tag_lst.append('VACC-B')
        elif token in ne_dict['DIS-B']:
            tag_lst.append('DIS-B')
        elif token in ne_dict['SYMP-B']:
            tag_lst.append('SYMP-B')
        else:
            tag_lst.append('O')
    
    token_tag_pairs = [token + ' ' + tag for token, tag in zip(token_lst, tag_lst)]  
    news_sents_tagged.append(' '.join(token_tag_pairs))
    
news_sents_tagged[:10]

100%|██████████| 4044/4044 [00:00<00:00, 24426.08it/s]


['전두환 O 전 O 대통령이 O 코로나19 O 화이자 VACC-B 백신을 O 접종한 O 뒤에 O 건강이 O 급격히 O 악화됐다는 O 전 O 씨 O 측 O 주장이 O 제기된 O 가운데 O 이에 O 대해 O 이재갑 O 한림대 O 감염내과 O 교수는 O 가당치도 O 않은 O 주장이라고 O 지적하고 O 나섰습니다 O . O',
 '이재갑 O 한림대 O 감염내과 O 교수는 O 25일 O CBS O 라디오 O 김현정의 O 뉴스쇼와의 O 인터뷰에서 O 전 O 씨 O 사망 O 원인으로 O 코로나 O 백신 O 부작용이 O 언급된 O 것에 O 대해 O 사실 O 가당치도 O 않은 O 주장이라며 O 최근에 O 혈액종양내과 O 교수님들이 O 여러 O 번 O 말씀드리긴 O 했는데 O 혈액암, O 백혈병하고 O 코로나 O 백신하고 O 관련이 O 없다고 O 선을 O 그었습니다 O . O',
 '이 O 교수는 O 전 O 세계적으로 O 관련이 O 증명된 O 사례는 O 전혀 O 없는 O 상황이고 O 다른 O 백신에서도 O 사례가 O 전혀 O 없다며 O 지금까지는 O 인과관계가 O 없다고 O 강조했습니다 O . O',
 '특히 O 전 O 전 O 대통령이 O 걸린 O 만성골수성백혈병 O 같은 O 경우는 O 원래 O 꽤 O 오래 O 전부터 O 시작되면서 O 서서히 O 시작되는 O 백혈병 O 중에 O 하나이기 O 때문에 O 그럴 O 가능성은 O 훨씬 O 더 O 떨어진다고 O 볼 O 수 O 있다고 O 전했습니다. O 백혈병이 O 급성으로 O 오기도 O 하지 O 않느냐는 O 진행자의 O 질문에는 O 급성 O 백혈병도 O 현재까지 O 인과관계가 O 증명된 O 사례는 O 없다고 O 전 O 씨의 O 사망 O 원인에 O 백신 O 부작용이 O 있다는 O 주장에 O 대해 O 다시 O 한 O 번 O 선을 O 그었습니다 O . O',
 '한편, O 전 O 씨는 O 지난 O 23일 O 오전 O 8시 O 40분쯤 O 연희동 O 자택에서 O 쓰러져 O 숨졌습니다 O . O',
 '이에 O 대해 O 전 O 씨 O 최측근 O

In [35]:
with open('../data/pre_tagged_sents.txt', 'w', encoding='utf-8') as f:
    for tagged_sent in news_sents_tagged:
        f.write(tagged_sent + '\n')

In [37]:
tagged_sents_df = pd.DataFrame(news_sents_tagged, columns=['tagged_sent'])
tagged_sents_df.to_csv('../data/pre_tagged_sents.csv', index=False, encoding='utf-8')